# GMO Forecasting


<p style="font-size:28px;">2. Analyzing GMO</p>

In [55]:
import math
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import statsmodels.api as sm

ANNUALIZATION_DAYS = 12

DATA_PATH = "data/gmo_analysis_data.xlsx"  

SIGNALS_SHEET = "signals"
RF_SHEET = "risk-free rate"
TOTAL_SHEET="total returns"

signals_df = pd.read_excel(DATA_PATH, sheet_name=SIGNALS_SHEET)
rf_df    = pd.read_excel(DATA_PATH, sheet_name=RF_SHEET)
total_df    = pd.read_excel(DATA_PATH, sheet_name=TOTAL_SHEET)



In [56]:
def subsample_func(dataset, start_date,end_date):
    df_subsample= dataset[(dataset['date'].dt.year>= start_date) & (dataset['date'].dt.year<= end_date)  ]
    df_subsample.drop(['date'],axis=1, inplace=True)
    df_subsample.dropna(inplace=True)
    return df_subsample

def ann_mean(x, periods=ANNUALIZATION_DAYS):
    return x.mean() * periods

def ann_vol(x, periods=ANNUALIZATION_DAYS):
    return x.std() * np.sqrt(periods)

def sharpe(x, periods=ANNUALIZATION_DAYS):
    mu = ann_mean(x, periods)
    sd = ann_vol(x, periods)
    return mu / sd


def stats_ff(dataset):

    annual_mean =ann_mean(dataset)
    annual_std =ann_vol(dataset)
    annual_sharpe =sharpe(dataset)


    stat_df = pd.DataFrame([{
    'Annualized Mean': annual_mean,
    'Annualized Vol': annual_std,
    'Sharpe': annual_sharpe
    }])

    
    return stat_df


def tail_risk(data):
    
    gmwax_returns = pd.Series(data).dropna()
    sorted_data=gmwax_returns.sort_values()
    var = sorted_data.quantile(0.05)

    cum = (1 + gmwax_returns).cumprod()
    peak = cum.cummax()
    dd = (cum - peak) / peak

    stat_df = pd.DataFrame([{
    'Min Return': gmwax_returns.min(),
    'Var': var,
    'Max Drawdown': dd.min()
    }])
    return stat_df

def reg_ols(op,ip, add_const=True):

    y=op

    x = sm.add_constant(ip) if add_const else x
    model = sm.OLS(y, x).fit()
    alpha = model.params["const"]
    beta  = model.params['SPY']
    r_sq  = model.rsquared
    return alpha*ANNUALIZATION_DAYS, beta, r_sq

**2.1 Performance (GMWAX). Compute mean, volatility, and Sharpe ratio for GMWAX over three samples:**

In [57]:
excess_df = pd.DataFrame()

for col in total_df.columns:
    if col == "date":
        continue
    excess_df[col]=total_df[col]-(rf_df.iloc[:,1] / ANNUALIZATION_DAYS)
excess_df['date']=total_df['date']

In [58]:
df1 =stats_ff(subsample_func(excess_df,1996,2011).iloc[:,1])
df2 =stats_ff(subsample_func(excess_df,2012,2025).iloc[:,1])
df3 =stats_ff(subsample_func(excess_df,1996,2025).iloc[:,1])

df_stats=pd.concat([df1,df2,df3])
df_stats.index=['1996-2011','2012-2025','1996-2025']
df_stats

,Annualized Mean,Annualized Vol,Sharpe
1996-2011,0.046422,0.110499,0.420110
2012-2025,0.049157,0.092661,0.530503
1996-2025,0.047730,0.102209,0.466986


*Has the mean, vol, and Sharpe changed much since the case?*</br>
The fund has improved substantially in the post-2011 period, returns are less negative, volatility is slightly lower, and the Sharpe ratio has improved by almost 1 full point—although overall risk-adjusted performance remains below zero.



**2.2 Tail risk (GMWAX). For all three samples, analyze extreme scenarios:</br>**

- **minimum return</br>**

- **5th percentile (VaR‑5th)</br>**

- **maximum drawdown (compute on total returns, not excess returns)**

In [59]:

tail_df1 =tail_risk(subsample_func(excess_df,1996,2011).iloc[:,1])
tail_df2 =tail_risk(subsample_func(excess_df,2012,2025).iloc[:,1])
tail_df3 =tail_risk(subsample_func(excess_df,1996,2025).iloc[:,1])
df_stats2=pd.concat([tail_df1,tail_df2,tail_df3])
df_stats2.index=['1996-2011','2012-2025','1996-2025']
df_stats2

,Min Return,Var,Max Drawdown
1996-2011,-0.149150,-0.044003,-0.306520
2012-2025,-0.115018,-0.039814,-0.225613
1996-2025,-0.149150,-0.041147,-0.306520


*(a) Does GMWAX have high or low tail‑risk as seen by these stats?*</br>
GMWAX exhibits very high tail-risk, with deep losses, fat left tails, and huge long-term drawdowns that highlight extreme downside risk.</br>
*(b) Does that vary much across the two subsamples?*</br>
GMWAX’s tail-risk is somewhat lower in the later period but remains very high in absolute terms. The underlying risk profile is broadly consistent across subsamples, with slight improvements.


**2.3 Market exposure (GMWAX). For all three samples, regress excess returns of GMWAX on excess returns of SPY:**
- **report estimated alpha, beta, and R²**

In [60]:

alpha1,beta1,r_sq1 =reg_ols(subsample_func(excess_df,1996,2011).iloc[:,1],subsample_func(excess_df,1996,2011).iloc[:,0],True)
alpha2,beta2,r_sq2 =reg_ols(subsample_func(excess_df,2012,2025).iloc[:,1],subsample_func(excess_df,2012,2025).iloc[:,0],True)
alpha3,beta3,r_sq3 =reg_ols(subsample_func(excess_df,1996,2025).iloc[:,1],subsample_func(excess_df,1996,2025).iloc[:,0],True)
df_Regression = pd.DataFrame({
    "Alpha(Annualized)": [alpha1,alpha2,alpha3],
    "Beta":  [beta1,beta2,beta3],
    "r_sq":  [r_sq1,r_sq2,r_sq3]
})
df_Regression

,Alpha(Annualized),Beta,r_sq
0,0.027000,0.542128,0.648686
1,-0.027362,0.566914,0.730904
2,0.002152,0.547452,0.675236


- *Is GMWAX a low‑beta strategy? has that changed since the case?*</br>
GMWAX is clearly a low-beta strategy, with beta aprrox 0.62. This characteristic has not changed meaningfully across subsamples or since the time of the case.</br>
- *Does GMWAX provide alpha? has that changed across subsamples?*</br>
GMWAX does not generate significant alpha in any period. Across both subsamples (1996–2011 and 2012–2025), alpha is consistently negative and very stable.

**2.4 Compare to GMGEX. Repeat items 1–3 for GMGEX**

In [61]:
df1 =stats_ff(subsample_func(excess_df,1996,2011).iloc[:,2])
df2 =stats_ff(subsample_func(excess_df,2012,2025).iloc[:,2])
df3 =stats_ff(subsample_func(excess_df,1996,2025).iloc[:,2])

df_stats=pd.concat([df1,df2,df3])
df_stats.index=['1996-2011','2012-2025','1996-2025']
df_stats

,Annualized Mean,Annualized Vol,Sharpe
1996-2011,-0.003823,0.147253,-0.025963
2012-2025,0.013182,0.228077,0.057794
1996-2025,0.004312,0.189982,0.022695


*Has the mean, vol, and Sharpe changed much since the case?*</br>
The mean return has improved substantially (less negative), volatility has increased meaningfully, and the Sharpe ratio, still negative, has improved a lot.

In [62]:
tail_df1 =tail_risk(subsample_func(excess_df,1996,2011).iloc[:,2])
tail_df2 =tail_risk(subsample_func(excess_df,2012,2025).iloc[:,2])
tail_df3 =tail_risk(subsample_func(excess_df,1996,2025).iloc[:,2])
df_stats2=pd.concat([tail_df1,tail_df2,tail_df3])
df_stats2.index=['1996-2011','2012-2025','1996-2025']
df_stats2

,Min Return,Var,Max Drawdown
1996-2011,-0.151592,-0.082292,-0.563989
2012-2025,-0.658863,-0.065603,-0.738304
1996-2025,-0.658863,-0.075737,-0.768086


*(a) Does GMGEX have high or low tail‑risk as seen by these stats?*</br>
Exhibits high tail-risk-worst returns, VaR-5, and maximum drawdown. After 2011, min return decreased significantly, VaR and Max Drawdown improved slightly.</br> 
*(b) Does that vary much across the two subsamples?*</br>
Tail-risk does vary across subsamples, but the variation depends on the metric.
Minimum returns become worse post-2011, indicating much more extreme single-period downside. VaR-5 and Maximum drawdowns improve slightly but remain damaging in both periods.

In [63]:

alpha1,beta1,r_sq1 =reg_ols(subsample_func(excess_df,1996,2011).iloc[:,2],subsample_func(excess_df,1996,2011).iloc[:,0],True)
alpha2,beta2,r_sq2 =reg_ols(subsample_func(excess_df,2012,2025).iloc[:,2],subsample_func(excess_df,2012,2025).iloc[:,0],True)
alpha3,beta3,r_sq3 =reg_ols(subsample_func(excess_df,1996,2025).iloc[:,2],subsample_func(excess_df,1996,2025).iloc[:,0],True)
df_Regression = pd.DataFrame({
    "Alpha(Annualized)": [alpha1,alpha2,alpha3],
    "Beta":  [beta1,beta2,beta3],
    "r_sq":  [r_sq1,r_sq2,r_sq3]
})
df_Regression

,Alpha(Annualized),Beta,r_sq
0,-0.031201,0.764237,0.725898
1,-0.097666,0.821257,0.253171
2,-0.060764,0.781633,0.398403


- *Is GMGEX a low‑beta strategy? has that changed since the case?*</br>
GMGEX is a low-beta strategy—with a beta of roughly 0.8 in all periods. This beta has remained remarkably stable across the two subsamples, indicating that the strategy’s market exposure has not changed since the time of the case.</br>
- *Does GMGEX provide alpha? has that changed across subsamples?*</br>
GMGEX does not provide significant alpha in either period. While the later sample shows slightly more negative alpha than the 1996–2011 period, the change is modest, and the fund’s underperformance remains essentially the same across subsamples.


*What are key differences between the two strategies?*</br>
GMWAX is a global, multi-asset macro allocation fund that generates returns by shifting across asset classes based on GMO’s 7-year valuation forecasts. GMGEX, in contrast, is a pure global equity strategy that relies on value-oriented stock selection rather than asset-class timing.